# Load Data

In [24]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from category_encoders.cat_boost import CatBoostEncoder
import category_encoders as ce

In [2]:
# Get parent working directory
cwd = Path.cwd()
parent_wd = cwd.parent
train_df = pd.read_csv(f"{str(parent_wd)}/data/DatiumTrain.rpt", sep="\t")
train_df.head(5)

/var/folders/s5/tr0c02592b79cz_d3swk4hvc0000gn/T/ipykernel_8541/4267819383.py:4: DtypeWarning: Columns (27,94) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(f"{str(parent_wd)}/data/DatiumTrain.rpt", sep="\t")


,Make,Model,MakeCode,FamilyCode,YearGroup,MonthGroup,SequenceNum,Description,CurrentRelease,ImportFlag,...,PrivateMax,NewPrice,Colour,Branch,SaleCategory,Sold_Date,Compliance_Date,Age_Comp_Months,KM,Sold_Amount
0,Holden,Commodore,HOLD,COMMODO,2008,0,0,VE Omega Sedan 4dr. Auto 4sp 3.6i,F,L,...,5500.0,34790.0,White,Perth (WA),Auction,2015-11-03 00:00:00.000,02/2008,93.0,227878.0,2000.0
1,Holden,Commodore,HOLD,COMMODO,1993,7,41,VR Executive Wagon 5dr. Auto 4sp 3.8i,F,L,...,2000.0,27978.0,Red,Belmore (NSW),Auction,2000-10-18 00:00:00.000,08/1993,86.0,153091.0,6800.0
2,Toyota,RAV4,TOYO,RAV4,2012,0,6,ACA33R MY12 CV Wagon 5dr Man 5sp 4x4 2.4i,F,L,...,15800.0,31990.0,040 - Glacier White (T),Sunshine (VIC),Dealer Only Auction,2014-02-05 00:00:00.000,10/2012,16.0,27374.0,22900.0
3,Holden,Commodore,HOLD,COMMODO,2007,0,11,VZ@VE MY07 Executive Wagon 5dr. Auto 4sp 3.6i,F,L,...,4800.0,35990.0,Quicksilver,Belmore (NSW),Auction,2011-01-10 00:00:00.000,01/2007,48.0,99452.0,10500.0
4,Toyota,Tarago,TOYO,TARAGO,2007,0,0,ACR50R GLi Wagon 8st 5dr Spts Auto 4sp 2.4i,F,L,...,12400.0,49490.0,Silver,Hobart (TAS),Special Fixed Price,2009-05-23 00:00:00.000,01/2007,28.0,44355.0,31320.0


# Transform and drop certain columns

In [3]:
target_col = "Sold_Amount"

In [4]:
# Drop rows with missing target 
train_df = train_df.dropna(subset=[target_col])

In [5]:
# Columns that the assesment brief explicitly mentioned not to use
not_to_use_cols = ['AvgWholesale', 'AvgRetail', 'GoodWholesale', 'GoodRetail', 'TradeMin', 'TradeMax', 'PrivateMax']

# Columns which have post-sale data identified in EDA phase
post_cols = ['NewPrice', 'Sold_Date']

# Columns identified in EDA that do not have much relevance to the vehicle
irrev_cols = ["SaleCategory", 'Branch']

# Columns to drop due to missingness of values
missingness_cols = ['BadgeSecondaryDescription',
 'WheelBaseConfig', 
 'Roofline',
 'PowerRPMFrom',
 'FreeScheduledService',
 'AltEngEngineType',
 'AltEngBatteryType',
 'AltEngCurrentType',
 'AltEngAmpHours',
 'AltEngVolts',
 'AltEngChargingMethod',
 'AltEngPower',
 'AltEngPowerFrom',
 'AltEngPowerTo',
 'AltEngTorque',
 'AltEngTorqueFrom',
 'AltEngTorqueTo',
 'AltEngDrive',
 'NormalChargeMins',
 'QuickChargeMins',
 'NormalChargeVoltage',
 'QuickChargeVoltage',
 'KMRangeElectricEng',
 'ElectricEngineLocation',
 'TopSpeedElectricEng',
 'MaxEthanolBlend',
 'TorqueRPMFrom',
 'EmissionStandard',
 'ExtraIdentification',
 'CO2Urban',
 'CO2ExtraUrban']

# Drop these columns
cols_to_drop = not_to_use_cols + post_cols + irrev_cols + missingness_cols
train_df = train_df.drop(columns=cols_to_drop)

In [6]:
# Transform Compliance Date column
train_df['Compliance_Date'] = pd.to_datetime(train_df['Compliance_Date'], format="%m/%Y",errors='coerce')
train_df['Compliance_Month'] = train_df['Compliance_Date'].dt.month
train_df['Compliance_Year'] = train_df['Compliance_Date'].dt.year

train_df = train_df.drop(columns=["Compliance_Date"])

In [7]:
# Log Transform Target
train_df["LogSoldAmount"] = np.log1p(train_df[target_col])

# Split Dataset by target and features

In [8]:
X = train_df.drop(columns=[target_col, "LogSoldAmount"])
y = train_df["LogSoldAmount"]

# Split features by numeric and categorical columns

In [9]:
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(include="object").columns.tolist()

# separate high cardinality columns and low ones as different encoding strategies will be used
high_card_cat_cols = [col for col in cat_cols if X[col].nunique() > 20]
low_card_cat_cols = list(set(cat_cols) - set(high_card_cat_cols))

# Impute Missing Values

In [10]:
# For the numerical columns, the standard practice would be to impute with the median value
num_imputer = SimpleImputer(strategy="median")
X[num_cols] = num_imputer.fit_transform(X[num_cols])

In [16]:
# For categorical columns, it would be best to set the null values as "Missing" or "Empty" etc.
cat_imputer = SimpleImputer(strategy="constant", fill_value='Missing')
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X[cat_cols] = X[cat_cols].astype(str)

# Split Train and Test sets

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Encoding Strategies

In [21]:
# One hot encoding for the features with low cardinality
onehot_encoder = OneHotEncoder(handle_unknown="ignore")
# Target encoding for the features with high cardinality, to avoid inflating the dataset.
target_encoder = ce.CountEncoder(cols=high_card_cat_cols)


In [22]:
X_train_enc = X_train.copy()
X_val_enc = X_val.copy()

In [ ]:
X_train_enc[high_card_cat_cols] = target_encoder.fit_transform(X_train[high_card_cat_cols], y_train)

X_val_enc[high_card_cat_cols] = target_encoder.transform(X_val[high_card_cat_cols])

# Feature Engineering Summary

It would be key to test out various encoding methods in experiments, so this will be done in the modelling phase